# Poppin 

In [136]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt 
import seaborn as sns
pd.options.mode.chained_assignment = None

### Scrape data


#### Scrape 40 most popular list

In [137]:
#https://stockx.com/sneakers/most-popular
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
mostpopularget = requests.get('https://stockx.com/sneakers/most-popular', headers=headers).text
mostpopularpage = BeautifulSoup(mostpopularget, 'html.parser')
sneakerdivs = mostpopularpage.findAll('div', class_='css-1ibvugw-GridProductTileContainer')
#class="css-1ibvugw-GridProductTileContainer"


In [138]:
#print(sneakerdivs[0].prettify())
mostpopular40 = pd.DataFrame()

for index, item in enumerate(sneakerdivs):
    #print(item.prettify())
    link = item.find('a')
    #print(link.attrs['href'])
    sneaker = item.find('p', class_='chakra-text css-3lpefb')
    #print(sneaker.string)
    newsneaker = pd.DataFrame({'Rank': (index+1), 'Sneaker':sneaker.string, 'Link': link.attrs['href']}, index=[index])
    mostpopular40 =  pd.concat([ mostpopular40, newsneaker],ignore_index=True)
    #mostpopular40 = mostpopular40.append({'Rank': (index+1), 'Sneaker':sneaker.string, 'Link': link.attrs['href']}, ignore_index=True, sort=False )
mostpopular40['Date in Most Popular'] = pd.to_datetime('today')

mostpopular40.to_csv('./scrape_pull/'+ str(pd.to_datetime('today').year)+str(pd.to_datetime('today').month)+str(pd.to_datetime('today').day)+'mostpopular40_list.csv')

#### Scrape details for 40 most popular

In [151]:
mostpopular40details = pd.DataFrame()
for index, item in mostpopular40.iterrows():
    #print(index)
    populartarget = requests.get('https://stockx.com'+item['Link'], headers=headers).text
    popularpage = BeautifulSoup(populartarget, 'html.parser')   
    sneakermodel = popularpage.find('h1', class_='chakra-heading css-146c51c')
    #print(sneakermodel.contents[0])
    sneakermodelsubname = popularpage.find('span', class_='chakra-heading css-xgzdo7')
    #print(sneakermodelsubname.string)
    sneakerdetails = popularpage.find_all('p', class_='chakra-text css-ziwxw9')
    #print(sneakerdetails[0].string) # StyleCode
    #print(sneakerdetails[1].string) # Colorway
    #print(sneakerdetails[2].string) # Retail Price
    #releasedate = sneakerdetails[3].string if len(sneakerdetails)==4 else np.nan

    lastsale = popularpage.find('p', class_='chakra-text css-xfmxd4')
    #print(lastsale.string)
    sneakerimage = popularpage.find('div', class_='css-1iec7mm')
    sneakerimageb = popularpage.find('div', class_='css-11a0fku')
    
    newsneaker = pd.DataFrame({'Rank': item['Rank'], 'Sneaker Model':sneakermodel.contents[0], 'Sneaker Name':sneakermodelsubname.string,'Colorway':sneakerdetails[1].string, 'Style Code':sneakerdetails[0].string, 'Release Date': (sneakerdetails[3].string if len(sneakerdetails)==4 else np.nan), 'Retail Price':sneakerdetails[2].string, 'Last Sale':lastsale.string, 'Image Link': (sneakerimage.contents[0].attrs['src'] if sneakerimage!=None else sneakerimageb.contents[1].attrs['src']) , 'Link': item['Link']}, index=[index])
    mostpopular40details =  pd.concat([ mostpopular40details, newsneaker],ignore_index=True)
mostpopular40details['Date in Most Popular'] = pd.to_datetime('today')

mostpopular40details.to_csv('./scrape_pull/'+ str(pd.to_datetime('today').year)+str(pd.to_datetime('today').month)+str(pd.to_datetime('today').day)+'mostpopular40_details.csv')

None
https://images.stockx.com/images/New-Balance-550-Conversations-Amongst-Us.jpg?fit=fill&bg=FFFFFF&w=480&h=320&fm=webp&auto=compress&dpr=2&trim=color&updated_at=1649781460&q=80


#### Daily Visualizations